In [ ]:
%cd /content
# %pip install -r /content/requirments.txt

In [ ]:
# Setup environment
import os
from dotenv import load_dotenv
from pprint import pprint as pp

load_dotenv()
print(os.environ['ZAPIER_WEBHOOK'])

import jax
jax.devices()

In [ ]:
import keras_nlp
# keras.config.set_floatx("float16")
# Model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_7b_en")
# gemma_lm.preprocessor.tokenizer = keras_nlp.models.GemmaTokenizer(
#     proto="/tokenizer/gemma_ua_ordered.model"
# )
# gemma_lm.load_weights("../chkpts/weights.h5")

In [ ]:
test_prompts_dev = [
    """Борщ це -""",
    """Крим - це територія"""
]

In [ ]:
with open('input_prompts.txt', 'r') as file:
    test_prompts = [line.rstrip('\n') for line in file]

def process_prompts():
    llm_outputs = []
    result_str = ""
    for prompt in test_prompts:
        result_str += f"PROMPT:\"{prompt}\"\n"
        output = gemma_lm.generate(prompt, max_length=96)
        llm_outputs.append(output)
        result_str += f"LLM:\n{output}\n\n"
    return result_str

In [ ]:
# Zapier snippet
import requests
from datetime import datetime
webhook_url = os.environ["ZAPIER_WEBHOOK"]
model_metadata = "blank metadata"
body = "blank body"
def post_to_slack():
    requests.post(webhook_url, json={'time': str(datetime.now().isoformat()), 'metadata': model_metadata, 'body': body, 'server': 'AMD mi300x'})

In [ ]:
model_metadata = f"gemma-7b base model vanilla"
body = process_prompts()
post_to_slack()

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_7b_en")
gemma_lm.preprocessor.tokenizer = keras_nlp.models.GemmaTokenizer(
    proto="/tokenizer/gemma_ua_ordered.model"
)
model_metadata = f"gemma-7b base model with custom UA tokenizer"
body = process_prompts()
post_to_slack()

In [ ]:
gemma_lm.load_weights("/ckpts/ckpt_meaninit_full_step_260000_loss_0.99.weights.h5")
model_metadata = f"gemma-7b base model with custom UA tokenizer and embd trained on 1B wiki-ua dataset"
body = process_prompts()
post_to_slack()

In [ ]:
folder_path = "/tmp/output"
file_names = os.listdir(folder_path)
print(len(file_names), sorted(file_names))
percents = [35, 50, 70, 85, 100, 130]
for elem in zip(percents, sorted(file_names)):
    print(f"{elem[0]}% of {elem[1]}")
    gemma_lm.load_weights(f"/tmp/output/{elem[1]}")
    model_metadata = f"{elem[0]}% of 1 epoch pretraining on 6B ua-wiki starting from trained embd"
    body = process_prompts()
    post_to_slack()
